In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:

variance = train_df.var(numeric_only=True)
zero_variance_columns = variance[variance == 0].index

train_df_cleaned = train_df.drop(columns=zero_variance_columns)
test_df_cleaned = test_df.drop(columns=zero_variance_columns)

In [ ]:
def label_encode_fit_transform(train_col, test_col):

    train_col = train_col.astype(str)
    test_col = test_col.astype(str)


    combined_data = pd.concat([train_col, test_col], axis=0)

    le = LabelEncoder()
    le.fit(combined_data)

    train_encoded = le.transform(train_col)
    test_encoded = pd.Series(test_col).apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)  # Replace unseen labels with -1

    return train_encoded, test_encoded


categorical_cols = train_df_cleaned.select_dtypes(include=['object']).columns

for col in categorical_cols:
    train_df_cleaned[col], test_df_cleaned[col] = label_encode_fit_transform(train_df_cleaned[col], test_df_cleaned[col])

In [ ]:
X = train_df_cleaned.drop(columns=['y', 'ID'])
y = train_df_cleaned['y']
test_X = test_df_cleaned.drop(columns=['ID'])


X = X.astype(float)
test_X = test_X.astype(float)


pca = PCA(n_components=100)
X_reduced = pca.fit_transform(X)
test_X_reduced = pca.transform(test_X)

In [ ]:

xgb_model = XGBRegressor()


xgb_model.fit(X_reduced, y)


y_pred = xgb_model.predict(test_X_reduced)


output = pd.DataFrame({'ID': test_df_cleaned['ID'], 'y': y_pred})
output.to_csv('submission.csv', index=False)

train_pred = xgb_model.predict(X_reduced)
mse = mean_squared_error(y, train_pred)
print(f'Mean Squared Error on Training Data: {mse}')

Mean Squared Error on Training Data: 7.156571875584457
